In [1]:
## USE THIS ONE
from numpy.linalg import inv
import numpy as np
import random

"""Note about the program: in (very) rare cases where the theta adjustment is extreme, the user will need to manually change theta to pi-theta before calculating
rotation and phi in the 'motor_angles' function."""

def Rot(r,p,t):
    """Find the matrix associated with rotations about the three axes"""
    Rr  = np.array([[np.cos(r),-np.sin(r),0],
                    [np.sin(r),np.cos(r),0],
                    [0,0,1]])
    Rt = np.array([[1,0,0],
                  [0,np.cos(t),-np.sin(t)],
                  [0,np.sin(t),np.cos(t)]])
    Rp = np.array([[np.cos(p),0,np.sin(p)],
                   [0,1,0],
                  [-np.sin(p),0,np.cos(p)]])
    return Rp@Rt@Rr

def motor_angles(Rf, Rs):
    """Find the necessary motor angles for the given orientation."""
    #Rf is the desired orientation
    #Rs is the sample orientation
    Rm = Rf@inv(Rs)

    theta = -np.arcsin(Rm[1][2]) #theta is in units of radians (tilt)
    phi = np.arcsin(Rm[0][2]/np.cos(theta)) #phi is in units of radians (R1)
    if Rm[2][2]/np.cos(theta)<0:
        phi = np.pi-phi
    rotation = np.arcsin(Rm[1][0]/np.cos(theta))
    if Rm[1][1]/np.cos(theta)<0:
        rotation = np.pi-rotation

    if rotation > np.pi:
        rotation -= 2*np.pi
    elif rotation < -np.pi:
        rotation += 2*np.pi

    if phi > np.pi:
        phi -= 2*np.pi
    elif phi < -np.pi:
        phi += 2*np.pi


    return rotation,phi,theta 

#Inputs
""""""
py_r = 4128*np.pi/180 #Sample rotation (z)
py_t = -162*np.pi/180 #Sample theta (x)
py_p = -0.5*np.pi/180 #Sample phi (y)

dx = -0.95
dy = 3.8
dz = 2.5 #Sample distance from origin (mm)

f_r = -68*np.pi/180 #Final rotation
f_p = 0*np.pi/180 #Final phi
f_t = 0*np.pi/180 #Final theta

""""""

Rs = Rot(py_r,py_p,py_t) #Sample orientation
Rf = Rot(f_r,f_p,f_t) #Final (desired) orientation

Output = motor_angles(Rf,Rs) 

print('R1 setting: '+str(Output[1]*180/np.pi)+' degrees')
print('Azimuthal setting: '+str(-Output[0]/(2*np.pi))+' rotations')
print('Tilt setting: '+str(Output[2]/(2*np.pi))+' rotations')

R = Rot(*Output)
D = np.array([dx,dy,dz])

Offset = R@D

print('')
print('X adjustment: '+str(-1/2**0.5*(Offset[0]-Offset[2]))+' mm')
print('Y adjustment: '+str(1/2**0.5*(Offset[0]+Offset[2]))+' mm')
print('Z adjustment: '+str(-Offset[1])+' mm') 
    

R1 setting: 165.19849499507436 degrees
Azimuthal setting: -0.15908773172695465 rotations
Tilt setting: -0.02880944400201822 rotations

X adjustment: -3.7963886744421296 mm
Y adjustment: 2.0835347348805007 mm
Z adjustment: -1.6848786428446307 mm
